In [101]:
from keras.models import Sequential
from keras.layers import Dense
import tensorflow as tf
import pandas as pd
import os
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import tensorflow.keras.layers as layers

In [102]:
root_directory = os.path.join(os.getcwd(), '../')
data_directory = os.path.join(root_directory ,'data')
pickle_name = 'After_preprocessing_raw_pickle'
pickle_path = os.path.join(data_directory, pickle_name)
raw_data = pd.read_pickle(pickle_path)
day_encoder = preprocessing.OneHotEncoder(sparse=False)
direction_encoder = preprocessing.OneHotEncoder(sparse=False)
merged_data = raw_data
merged_data = merged_data.drop(columns=['Report Date'])

In [103]:
merged_data['Route'] = merged_data['Route'].astype('int64')

In [104]:
merged_data.dtypes

Route          int64
Direction     object
hour           int64
Day           object
Year           int64
Month          int64
Daym           int64
target       float64
dtype: object

In [105]:
categorical_col_int = ['hour', 'Month', 'Daym', 'Route', 'Year']
categorical_col_str = ['Direction', 'Day']

In [106]:
train, test = train_test_split(merged_data, test_size=0.3)
train, val = train_test_split(train, test_size=0.3)


In [107]:
def df_to_datset(dataframe, shuffle=True, batch_size=1280):
    df = merged_data.copy()
    labels = df.pop('target')
    df = {key: value[:, tf.newaxis] for key, value in merged_data.items()}
    ds = tf.data.Dataset.from_tensor_slices((df, labels))
    ds = ds.batch(batch_size)
    ds = ds.prefetch(batch_size)
    return ds

In [108]:
train_ds = df_to_datset(train)
test_ds = df_to_datset(test)
val_ds = df_to_datset(val)

/opt/anaconda3/envs/test/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  after removing the cwd from sys.path.


In [109]:
def get_category_encoding_layer(name, dataset, dtype, max_tokens=None):
    if dtype == 'string':
        index = layers.StringLookup(max_tokens=max_tokens)
    else:
        index = layers.IntegerLookup(max_tokens = max_tokens)
    feature_ds = dataset.map(lambda x, y: x[name])
    index.adapt(feature_ds)
    encoder = layers.CategoryEncoding(num_tokens=index.vocabulary_size())
    return lambda feature: encoder(index(feature))

In [110]:
all_inputs = []
encoded_features = []

# for header in categorical_col_int:
#     categorical_int_col = tf.keras.Input(shape=(1,), name=header, dtype='int64', )
#     encoding_layer = get_category_encoding_layer(name=header, dataset=train_ds, dtype='int64')
#     encoded_categorical_col = encoding_layer(categorical_col_int)
#     all_inputs.append(categorical_int_col)
#     encoded_features.append(encoded_categorical_col)

for header in categorical_col_str:
    categorical_str_col = tf.keras.Input(shape=(1,), name=header, dtype='string')
    encoding_layer = get_category_encoding_layer(name=header, dataset=train_ds, dtype='string')
    encoded_categorical_col = encoding_layer(categorical_str_col)
    all_inputs.append(categorical_str_col)
    encoded_features.append(encoded_categorical_col)

In [111]:
for header in categorical_col_int:
    categorical_int_col = tf.keras.Input(shape=(1,), name=header, dtype='int64')
    encoding_layer = get_category_encoding_layer(name=header, dataset=train_ds, dtype='int64', max_tokens=5)
    encoded_categorical_col = encoding_layer(categorical_int_col)
    all_inputs.append(categorical_int_col)
    encoded_features.append(encoded_categorical_col)

In [112]:
all_features = tf.keras.layers.concatenate(encoded_features)
x = tf.keras.layers.Dense(32, activation="relu")(all_features)
x = tf.keras.layers.Dropout(0.5)(x)
output = tf.keras.layers.Dense(1)(x)

model = tf.keras.Model(all_inputs, output)

In [113]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=["accuracy"])

In [115]:
callback = tf.keras.callbacks.EarlyStopping(min_delta=0.01,monitor='loss')

In [116]:
model.fit(train_ds, epochs=10, validation_data=val_ds, callbacks=[callback])


Epoch 1/10


/opt/anaconda3/envs/test/lib/python3.7/site-packages/keras/engine/functional.py:566: UserWarning: Input dict contained keys ['target'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)


2401/2401 [==============================] - 38s 15ms/step - loss: 0.0917 - accuracy: 0.9821 - val_loss: 0.1045 - val_accuracy: 0.9825
Epoch 2/10
2401/2401 [==============================] - 37s 15ms/step - loss: 0.0771 - accuracy: 0.9825 - val_loss: 0.0959 - val_accuracy: 0.9825


In [117]:
loss, accuracy = model.evaluate(test_ds)
print("Accuracy", accuracy)

2401/2401 [==============================] - 18s 8ms/step - loss: 0.0959 - accuracy: 0.9825
Accuracy 0.9824689626693726


In [118]:
test_ds

<PrefetchDataset element_spec=({'Route': TensorSpec(shape=(None, 1), dtype=tf.int64, name=None), 'Direction': TensorSpec(shape=(None, 1), dtype=tf.string, name=None), 'hour': TensorSpec(shape=(None, 1), dtype=tf.int64, name=None), 'Day': TensorSpec(shape=(None, 1), dtype=tf.string, name=None), 'Year': TensorSpec(shape=(None, 1), dtype=tf.int64, name=None), 'Month': TensorSpec(shape=(None, 1), dtype=tf.int64, name=None), 'Daym': TensorSpec(shape=(None, 1), dtype=tf.int64, name=None), 'target': TensorSpec(shape=(None, 1), dtype=tf.float64, name=None)}, TensorSpec(shape=(None,), dtype=tf.float64, name=None))>